In [11]:
TEMP_RES.gestures[0][0].category_name

[[Category(index=-1, score=0.5797708630561829, display_name='', category_name='none')]]

In [1]:
import time

import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

recognizer_model_path = 'models/gesture_recognizer.task'
LABELS = ['call', 'dislike', 'fist', 'like', 'mute', 'ok', 'one', 'palm', 'peace', 'rock', 'stop', 'stop_inverted']
# LABELS = ['🤙', '👎', '✊', '👍', '🤐', '👌', '☝', '🖐', '✌', '🤘', '✋', '🤚']

BaseOptions = mp.tasks.BaseOptions
GestureRecognizer = mp.tasks.vision.GestureRecognizer
GestureRecognizerOptions = mp.tasks.vision.GestureRecognizerOptions
GestureRecognizerResult = mp.tasks.vision.GestureRecognizerResult
VisionRunningMode = mp.tasks.vision.RunningMode
capture = cv2.VideoCapture(0)

# Create a gesture recognizer instance with the live stream mode:
def print_result(result: GestureRecognizerResult, output_image: mp.Image, timestamp_ms: int):
    if result.hand_landmarks:
        # gesture_res = result.gestures[0].category_name
        global TEMP_RES
        TEMP_RES = result
        for gesture in result.gestures:
            for hand in gesture:
                category_name = hand.category_name
                score = hand.score
        print(f'{timestamp_ms}: {category_name}, score')


options = GestureRecognizerOptions(
    base_options=BaseOptions(model_asset_path=recognizer_model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    num_hands=1,
    result_callback=print_result)

with GestureRecognizer.create_from_options(options) as recognizer:
    while capture.isOpened():
        ret, frame = capture.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        frame_timestamp_ms = int(time.time() * 1000)
        recognizer.recognize_async(mp_image, frame_timestamp_ms)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        cv2.imshow('Webcam', image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        
capture.release()
cv2.destroyAllWindows()

print(TEMP_RES)


1709844951746: GestureRecognizerResult(gestures=[[Category(index=-1, score=0.8559032082557678, display_name='', category_name='stop')]], handedness=[[Category(index=1, score=0.6443670988082886, display_name='Left', category_name='Left')]], hand_landmarks=[[NormalizedLandmark(x=0.6696726679801941, y=0.7877281904220581, z=2.4602070425316924e-07, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.600643515586853, y=0.7845807671546936, z=-0.02723585069179535, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5431705117225647, y=0.7312086224555969, z=-0.042196329683065414, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5083299875259399, y=0.6727773547172546, z=-0.05406690016388893, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.4787479043006897, y=0.6310780048370361, z=-0.06523295491933823, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.5655571222305298, y=0.6131069660186768, z=-0.028197983279824257, visibility=0.0, presence=0.0), NormalizedLandmark(x=0.53171372

In [13]:
from models import KeyPointClassifier
import copy 
import itertools
import tensorflow as tf
import numpy as np

landmark_list = [0.6641262173652649, 0.7233468294143677, 2.483685932475055e-07, 0.6084092855453491, 0.6979326009750366, -0.015598497353494167, 0.5637728571891785, 0.6345027685165405, -0.021485067903995514, 0.5399306416511536, 0.575805127620697, -0.027300065383315086, 0.5146738886833191, 0.5334270596504211, -0.03330542892217636, 0.5886807441711426, 0.5339487195014954, -0.0057119629345834255, 0.5640174150466919, 0.4607226550579071, -0.020116688683629036, 
0.5498967170715332, 0.4148087501525879, -0.03349391743540764, 0.5413908362388611, 0.3743138909339905, -0.043615296483039856, 0.6222752928733826, 0.5165725946426392, -0.011697640642523766, 0.6089351177215576, 0.4260062277317047, 
-0.022712308913469315, 0.6021220088005066, 0.3694627285003662, -0.034752532839775085, 0.597951352596283, 0.32305794954299927, -0.044126953929662704, 0.6556739807128906, 0.5185085535049438, -0.021214893087744713, 0.6478569507598877, 0.4312754273414612, -0.03423449769616127, 0.6435176134109497, 0.37731853127479553, -0.04411550238728523, 0.6390867233276367, 0.331074059009552, -0.051843732595443726, 0.6896733641624451, 0.5360729694366455, -0.032618217170238495, 0.6908791661262512, 0.47086232900619507, -0.04455866292119026, 0.6920442581176758, 0.4291669428348541, -0.04890189692378044, 0.6919578313827515, 0.39215558767318726, -0.05228421092033386]

model_path='models/keypoint_classifier.tflite'
num_threads=1

interpreter = tf.lite.Interpreter(model_path=model_path,
                                        num_threads=num_threads)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_details_tensor_index = input_details[0]['index']
interpreter.set_tensor(
            input_details_tensor_index,
            np.array([landmark_list], dtype=np.float32))
interpreter.invoke()

output_details_tensor_index = output_details[0]['index']
result = interpreter.get_tensor(output_details_tensor_index)
result_index = np.argmax(np.squeeze(result))

print(result_index)





10


In [ ]:
import mediapipe as mp
import cv2
import time

model_path = 'models/hand_landmarker.task'

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode
capture = cv2.VideoCapture(0)

# Create a hand landmarker instance with the live stream mode:
def print_result(result: HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
    if result.hand_landmarks:
        print(f'hand landmarker result {timestamp_ms}: {result.hand_landmarks}')

options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.LIVE_STREAM,
    result_callback=print_result)

with HandLandmarker.create_from_options(options) as landmarker:
    while capture.isOpened():
        ret, frame = capture.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        frame_timestamp_ms = int(time.time() * 1000)
        landmarker.detect_async(mp_image, frame_timestamp_ms)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        cv2.imshow('Webcam', image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        
capture.release()
cv2.destroyAllWindows()

In [ ]:
import mediapipe as mp
import cv2

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

capture = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
    while capture.isOpened():
        ret, frame = capture.read()
        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detected_image = hands.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if detected_image.multi_hand_landmarks:
            for hand_lms in detected_image.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_lms,
                                            mp_hands.HAND_CONNECTIONS,
                                            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                                                color=(255, 0, 255), thickness=4, circle_radius=2),
                                            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(
                                                color=(20, 180, 90), thickness=2, circle_radius=2)
                                            )
    
        cv2.imshow('Webcam', image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

capture.release()
cv2.destroyAllWindows()

<module 'mediapipe.python.solutions.hands' from 'c:\\Users\\Ian\\git\\553.806_Capstone_HandGesture\\.venv\\Lib\\site-packages\\mediapipe\\python\\solutions\\hands.py'>
